## Train sea lion classifier with a convnet

In [1]:
INPUT_DIR = '../../output/kaggle-sea-lion/02/'
OUTPUT_DIR = '../../output/kaggle-sea-lion/03/'
IMAGE_DIMS = (148,148,3)

MODEL_FILE = OUTPUT_DIR + 'sea-lion-classifier'

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import sklearn
import os
import glob

from modules.logging import logger
import modules.utils as utils
from modules.utils import Timer
import modules.logging
import modules.cnn as cnn

## Training

### Prepare output dir

In [3]:
utils.mkdirs(OUTPUT_DIR, recreate=False)
modules.logging.setup_file_logger(OUTPUT_DIR + 'out.log')
logger.info('Dir ' + OUTPUT_DIR + ' created')

load_model_file = None
if(os.path.isfile(MODEL_FILE + '.index')):
    load_model_file = MODEL_FILE
    logger.info('Found previous model file. It will be loaded on cnn network as its initial state. ' + load_model_file)

2017-04-02 20:30:04,314 INFO Dir ../../output/kaggle-sea-lion/03/ created
2017-04-02 20:30:04,316 INFO Found previous model file. It will be loaded on cnn network as its initial state. ../../output/kaggle-sea-lion/03/sea-lion-classifier


### Prepare CNN model

In [4]:
logger.info('Prepare CNN for training')
network = cnn.net_alexnet_lion(IMAGE_DIMS)
model = cnn.prepare_cnn_model(network, OUTPUT_DIR, model_file=load_model_file)

2017-04-02 20:30:04,323 INFO Prepare CNN for training
2017-04-02 20:30:04,449 INFO Prepare CNN
2017-04-02 20:30:04,450 INFO Preparing output dir
2017-04-02 20:30:04,451 INFO Initializing network...
2017-04-02 20:30:07,016 INFO Network initialized
2017-04-02 20:30:07,018 INFO Load previous training...
2017-04-02 20:30:34,641 INFO Model loaded


### Train model

In [5]:
dataset_path = INPUT_DIR + utils.dataset_name('lion-patches', IMAGE_DIMS)

with h5py.File(dataset_path, 'r') as hdf5:
    X,Y = utils.dataset_xy_range(hdf5, 0, 0.9)
    logger.info('X shape ' + str(X.shape))
    logger.info('Y shape ' + str(Y.shape))

    logger.info('Starting CNN training...')
    model.fit(X, Y, 
        validation_set=0.2,
        shuffle=True, 
        batch_size=96, 
        n_epoch=10,
        show_metric=True,
        snapshot_epoch=False,
        run_id='sea_lion_classifier')

model.save(MODEL_FILE)
logger.info("Network trained and saved as " + MODEL_FILE)

Training Step: 79  | total loss: 1.11453 | time: 164.598s
| Momentum | epoch: 010 | loss: 1.11453 - acc: 0.5424 -- iter: 672/680
Training Step: 80  | total loss: 1.11619 | time: 180.698s
| Momentum | epoch: 010 | loss: 1.11619 - acc: 0.5380 | val_loss: 1.00900 - val_acc: 0.6059 -- iter: 680/680
--
INFO:tensorflow:/notebooks/output/kaggle-sea-lion/03/sea-lion-classifier.tfl is not in all_model_checkpoint_paths. Manually adding it.


2017-04-02 19:52:13,511 INFO /notebooks/output/kaggle-sea-lion/03/sea-lion-classifier.tfl is not in all_model_checkpoint_paths. Manually adding it.
2017-04-02 19:52:13,630 INFO Network trained and saved as sea-lion-classifier.tfl!


### Evaluate results

In [5]:
logger.info('Evaluate dataset')
dataset_path = INPUT_DIR + utils.dataset_name('lion-patches', IMAGE_DIMS)

with h5py.File(dataset_path, 'r') as hdf5:
    X,Y = utils.dataset_xy_range(hdf5, 0.9, 1)
    cnn.evaluate_dataset(X, Y, model, batch_size=24, detailed=True)

2017-04-02 20:30:34,649 INFO Evaluate dataset
2017-04-02 20:30:41,233 INFO Accuracy: [0.58947368546536094]
2017-04-02 20:30:41,235 INFO Confusion matrix


ValueError: Can't handle mix of multilabel-indicator and multiclass-multioutput

In [71]:
from sklearn import metrics
from sklearn import preprocessing
import itertools

with h5py.File(dataset_path, 'r') as hdf5:
    X,Y = utils.dataset_xy_range(hdf5, 0.9, 1)

    acc = model.evaluate(X, Y, batch_size=24)
    logger.info('Accuracy: ' + str(acc))
    
    logger.info('Confusion matrix')
    Y_pred = model.predict_label(X[0:1,:])
    Y_prob = model.predict(X[0:1,:])

    print(Y[0])
    print(Y_pred)
    print(Y_prob)
    
    #we only need the highest probability guess
    Y_pred = np.flip(Y_pred, 1)
    Y_pred = Y_pred[:,0]
    
    #convert from categorical to label
    lb = preprocessing.LabelBinarizer()
    lb.fit(np.array(range(5)))
    Y = lb.inverse_transform(Y)
    
    print(Y)
    print(Y_pred)
    
    print('Kappa score (was this luck?): ' + str(metrics.cohen_kappa_score(Y, Y_pred)))
    
    cm = metrics.confusion_matrix(Y, Y_pred)
    print(cm)
    
    normalize = False
    
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    class_labels = []
    print('range ' + str(len(cm)))
    class_labels = ["{:d}".format(x) for x in range(len(cm))]
    tick_marks = np.arange(len(class_labels))
    plt.xticks(tick_marks, class_labels, rotation=45)
    plt.yticks(tick_marks, class_labels)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.nan_to_num(cm)
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


2017-04-02 22:34:32,774 INFO Accuracy: [0.58947368546536094]
2017-04-02 22:34:32,776 INFO Confusion matrix


[ 0.  0.  0.  1.  0.]
[[1 3 0 4 2]]
[[0.06448108702898026, 0.021181099116802216, 0.22801533341407776, 0.09014074504375458, 0.5961816906929016]]
[3 2 0 2 2 2 4 2 4 4 2 2 3 2 2 4 2 2 3 2 4 2 4 2 2 2 4 2 0 4 2 2 4 2 4 2 4
 2 2 4 2 2 2 2 4 2 2 4 2 2 2 2 4 4 2 2 4 4 4 2 2 4 2 2 2 4 2 2 2 3 4 2 2 2
 2 4 4 2 4 2 4 4 2 2 0 2 2 2 4 4 4 2 3 2 4]
[2]


ValueError: Found input variables with inconsistent numbers of samples: [95, 1]

In [66]:
from tflearn.data_utils import to_categorical

y = to_categorical([2,4,0,2,1], nb_classes=5)
print(y)

[[ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.]]
